In [9]:
import json
import pandas as pd

In [10]:
df = pd.read_csv('../results/.csv')
with open('../../dataset/test.json') as f:
    data = json.load(f)

In [11]:
idx = 23139
test_annotations = []
for i in range(len(df)):
    pred = df.iloc[i]['PredictionString']
    img_id = df.iloc[i]['image_id']
    pred_list = pred.split(' ')
    if float(pred_list[1]) < 0.4 or float(pred_list[1]) >= 1:
        x1 = float(pred_list[2])
        y1 = float(pred_list[3])
        x2 = float(pred_list[4])
        y2 = float(pred_list[5])
        w = x2 - x1
        h = y2 - y1
        img_id = img_id.replace('test/', '')
        img_id = img_id.replace('.jpg', '')
        annotation = {
                'image_id' : int(img_id) + 4883,
                'category_id' : int(pred_list[0]),
                'area' : w*h,
                'bbox' : [
                    x1, x2, w, h
                    ],
                'iscrowd' : 0,
                'id': idx
        }
        test_annotations.append(annotation)
        idx += 1
    for p in range(len(pred_list) // 6):
        pred_one = pred_list[p*6:(p+1)*6]
        
        if float(pred_one[1]) >= 0.4:
            x1 = float(pred_one[2])
            y1 = float(pred_one[3])
            x2 = float(pred_one[4])
            y2 = float(pred_one[5])
            w = x2 - x1
            h = y2 - y1
            img_id = img_id.replace('test/', '')
            img_id = img_id.replace('.jpg', '')
            annotation = {
                'image_id' : int(img_id) + 4883,
                'category_id' : int(pred_one[0]),
                'area' : w*h,
                'bbox' : [
                    x1, x2, w, h
                    ],
                'iscrowd' : 0,
                'id': idx
            }
            test_annotations.append(annotation)
            idx += 1


for i in data['images']:
    i['id'] = int(i['id']) + 4883

In [12]:
test_data = {
        'info' : data['info'],
        'licenses': data['licenses'],
        'images': data['images'],
        'annotations': test_annotations,
        'categories': data['categories'],
    }

In [13]:
with open('../../dataset/pseudo_labeling.json', 'w') as f:
    json.dump(test_data, f, indent=4)

In [15]:

with open('../../dataset/train_fold_4.json') as f:
    train_data = json.load(f)
with open('../../dataset/pseudo_labeling.json') as f:
    test_data = json.load(f)

data= {
        'info' : train_data['info'],
        'licenses' : train_data['licenses'],
        'images' : train_data['images'] + test_data['images'],
        'categories' : train_data['categories'],
        'annotations' : train_data['annotations'] + test_data['annotations']
        }

with open('../../dataset/pseudo_label.json', 'w') as f:
    json.dump(data, f, indent=4)
